In [15]:
import pyaudio

def record_audio():
    CHUNK = 2048  # Number of audio frames per buffer
    FORMAT = pyaudio.paInt16  # Format for audio input
    CHANNELS = 1  # Mono
    RATE = 16000  # Sampling rate

    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    
    print("Recording...")
    return stream


In [16]:
import whisper

def transcribe_audio_whisper(stream):
    model = whisper.load_model("base")  # Load Whisper model
    audio_data = b""

    while True:
        data = stream.read(1024)
        audio_data += data

        # Transcribe when audio reaches a certain size
        if len(audio_data) > RATE * 5:  # 5 seconds of audio
            result = model.transcribe(audio_data)
            print(result["Recorded"])
            audio_data = b""  # Reset the buffer


In [18]:
from google.cloud import speech_v1p1beta1 as speech

def transcribe_audio_google(stream):
    client = speech.SpeechClient()
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )
    streaming_config = speech.StreamingRecognitionConfig(config=config)

    audio_generator = (stream.read(1024) for _ in range(0, int(RATE / 1024)))
    requests = (speech.StreamingRecognizeRequest(audio_content=chunk) for chunk in audio_generator)

    responses = client.streaming_recognize(streaming_config, requests)
    for response in responses:
        for result in response.results:
            print("Transcript:", result.alternatives[0].transcript)


In [13]:
import queue
import threading

audio_queue = queue.Queue()

def audio_callback(stream):
    while True:
        data = stream.read(CHUNK)
        audio_queue.put(data)

def transcription_worker():
    audio_buffer = b""
    while True:
        if not audio_queue.empty():
            audio_buffer += audio_queue.get()
        if len(audio_buffer) >= RATE * 5:  # 5 seconds of audio
            # Transcribe and reset the buffer
            result = model.transcribe(audio_buffer)
            print(result["text"])
            audio_buffer = b""

# Run audio processing and transcription in parallel
threading.Thread(target=audio_callback, args=(stream,)).start()
threading.Thread(target=transcription_worker).start()


Exception in thread Thread-9 (audio_callback):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-10 (transcription_worker):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/q3/p5td8yjd4zl74d8v3hztzpwh0000gn/T/ipykernel_42642/3530458797.py", line 8, in audio_callback
    self.run()
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1012, in run
NameError: name 'CHUNK' is not defined
    self._target(*self._args, **self._kwargs)
  File "/var/folders/q3/p5td8yjd4zl74d8v3hztzpwh0000gn/T/ipykernel_42642/3530458797.py", line 16, in transcription_worker
NameError: name 'RATE' is not defined


In [19]:
if __name__ == "__main__":
    stream = record_audio()

    # For Whisper:
    transcribe_audio_whisper(stream)

    # For Google Speech-to-Text:
    # transcribe_audio_google(stream)


Recording...


OSError: [Errno -9981] Input overflowed